<a href="https://colab.research.google.com/github/meerab-17/News-Topic-Classifier-Using-BERT/blob/main/newsTopicClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem Statement: To build an intelligent system that can automatically classify news headlines into predefined categories such as World, Sports, Business, and Sci/Tech.

In [1]:
from datasets import load_dataset
from transformers import BertTokenizerFast

dataset = load_dataset("ag_news")
print("loaded")
print(dataset)

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=64)

tokenized_dataset = dataset.map(tokenize, batched=True)
print("Tokenization complete!")
print(tokenized_dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

Tokenization complete!
DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7600
    })
})


In [2]:
from transformers import BertForSequenceClassification, TrainingArguments, Trainer
import torch

tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

train_dataset = tokenized_dataset["train"]
test_dataset = tokenized_dataset["test"]

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_steps=50
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: meerabfarooq50 (meerabfarooq50-lums) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,0.789700
100,0.421700
150,0.414600
200,0.607400
250,0.505700
300,0.454600
350,0.443800
400,0.430600
450,0.450800
500,0.506700


TrainOutput(global_step=15000, training_loss=0.29260826063156126, metrics={'train_runtime': 3252.9433, 'train_samples_per_second': 36.89, 'train_steps_per_second': 4.611, 'total_flos': 3946736701440000.0, 'train_loss': 0.29260826063156126, 'epoch': 1.0})

In [3]:
#model evaluation
from sklearn.metrics import accuracy_score, f1_score, classification_report

# predictions
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(axis=1)
labels = predictions.label_ids

# metrics calculation
acc = accuracy_score(labels, preds)
f1 = f1_score(labels, preds, average='weighted')

print(f"Test Accuracy: {acc:.4f}")
print(f"Test F1-score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(labels, preds))

Test Accuracy: 0.9438
Test F1-score: 0.9438

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.95      0.95      1900
           1       0.99      0.99      0.99      1900
           2       0.91      0.92      0.92      1900
           3       0.92      0.92      0.92      1900

    accuracy                           0.94      7600
   macro avg       0.94      0.94      0.94      7600
weighted avg       0.94      0.94      0.94      7600



In [4]:
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')

This task involves building a text classification model using the AG News dataset. A pre-trained BERT model (bert-base-uncased) is fine-tuned to predict the topic of a news headline. The process includes data preprocessing, model training, evaluation using accuracy and F1-score, and deployment via a simple Streamlit interface for live user interaction.

